<a href="https://colab.research.google.com/github/alejogiley/Hack7/blob/master/Rui_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Anaconda

In [1]:
!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local

!conda install -q -y --prefix /usr/local -c omnia --no-update-deps pdbfixer=1.4
!conda install -q -y --prefix /usr/local -c conda-forge --no-update-deps xgboost=0.6a2
!conda install -q -y --prefix /usr/local -c rdkit --no-update-deps rdkit=2017.09.1
!conda install -q -y --prefix /usr/local -c deepchem --no-update-deps  deepchem-gpu=2.1.0

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')
#ignore warning about PYTHONPATH

--2019-08-16 14:45:22--  https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 577996269 (551M) [application/x-sh]
Saving to: ‘Anaconda3-5.1.0-Linux-x86_64.sh’

Anaconda3-5.1.0-Lin 100%[===================>] 551.22M   182MB/s    in 3.0s    

2019-08-16 14:45:25 (182 MB/s) - ‘Anaconda3-5.1.0-Linux-x86_64.sh’ saved [577996269/577996269]

PREFIX=/usr/local
installing: python-3.6.4-hc3d631a_1 ...
Python 3.6.4 :: Anaconda, Inc.
installing: ca-certificates-2017.08.26-h1d4fec5_0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: intel-openmp-2018.0.0-hc7b2577_8 ...
installing: libgcc-ng-7.2.0-h7cc24e2_2 ...
installing: libgfortran-ng-7.2.0-h9f7466a_2 ...
installing: libstdcxx-ng-7.2.0-h7a57d05_2 ...
installing: bzip2-1.0.6-h9a117a

In [2]:
import os
import pandas as pd
import numpy as np

import deepchem as dc
import sklearn as sk

from rdkit import RDConfig
from rdkit import Chem

from rdkit.Chem import PandasTools
from rdkit.Chem import ChemicalFeatures
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Crippen import MolLogP
from rdkit.Chem.Descriptors import ExactMolWt

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Load Dataset
az = pd.read_csv('/content/drive/My Drive/hack7/AZ_Pyrrolamides_4OE.csv', encoding='GBK')

In [34]:
az['IC50'] = az['SAU Gyr IC50 (礛)'].str.strip('>').str.strip('<').astype(float)
az['Censorship'] = 'normal'
az['Censorship'].loc[az['SAU Gyr IC50 (礛)'].str.contains('>')] = '>'
az['Censorship'].loc[az['SAU Gyr IC50 (礛)'].str.contains('<')] = '<'
az['logIC50'] = np.log(az['IC50'])
az.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 19 columns):
External_ID                         981 non-null object
Molecule SMILES                     981 non-null object
First Sample Reg. Date              981 non-null object
SAU Gyr IC50 (礛)                    981 non-null object
SAU Gyr Ki (礛)                      444 non-null object
Sau516 MIC (礸/ml)                   779 non-null object
Sau516  MIC (礛)                     709 non-null object
Mean Human Prot binding (% free)    596 non-null object
Mean LogD                           553 non-null object
Determination of pKa HA1            13 non-null float64
Determination of pKa HA2            15 non-null float64
Determination of pKa B1             12 non-null float64
Determination of pKa B2             1 non-null float64
Sirius pKa                          9 non-null object
IC50                                981 non-null float64
Censorship                          981 non-null object
m

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Getting more feature for mulecular

In [22]:
# Get Descriptors
az['mol'] = az['Molecule SMILES'].apply(lambda x: Chem.MolFromSmiles(x)) 
print(type(az['mol'][1])) 

<class 'rdkit.Chem.rdchem.Mol'>


In [0]:
# Define descriptor function
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
descriptors = list(np.array(Descriptors._descList)[:,0])
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptors)

def computeDescriptors(mol, calculator):
    res = np.array(calculator.CalcDescriptors(mol))
    if not np.all(np.isfinite(res)):
        return None  #make is easier to identify problematic molecules (.e.g infity descriptor values) later 
    return res 
  
# Making descriptors and calculator list
descriptors = list(np.array(Descriptors._descList)[:,0])
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptors) 


In [0]:
# Making descriptors and calculator list
descriptors = list(np.array(Descriptors._descList)[:,0])
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptors) 
#PandasTools.AddMoleculeColumnToFrame(frame=az, smilesCol='Molecule SMILES', molCol='mol_2') 

In [0]:
descrip = az[[ "mol"]].copy()
#Mapping descriptors with molecules
descrip['Descriptors'] = descrip['mol'].map(lambda x: computeDescriptors(x, calculator)) 

In [30]:
problems = descrip["Descriptors"].isnull()
print("{} molecules failed to get descriptors".format(problems.sum())) 

0 molecules failed to get descriptors


In [0]:
DES = descrip["Descriptors"].apply(pd.Series)
DES = DES.rename(columns = lambda x : descriptors[x])

In [44]:
az.apply(lambda x: sum(x.isnull()))

External_ID                           0
Molecule SMILES                       0
First Sample Reg. Date                0
SAU Gyr IC50 (礛)                      0
SAU Gyr Ki (礛)                      537
Sau516 MIC (礸/ml)                   202
Sau516  MIC (礛)                     272
Mean Human Prot binding (% free)    385
Mean LogD                           428
Determination of pKa HA1            968
Determination of pKa HA2            966
Determination of pKa B1             969
Determination of pKa B2             980
Sirius pKa                          972
IC50                                  0
Censorship                            0
mol                                   0
mol_2                                 0
logIC50                               0
dtype: int64

In [53]:
df_final = pd.concat([az[["Censorship"]], DES], axis=1)
df_final.apply(lambda x: sum(x.isnull()))

Censorship                  0
MaxEStateIndex              0
MinEStateIndex              0
MaxAbsEStateIndex           0
MinAbsEStateIndex           0
qed                         0
MolWt                       0
HeavyAtomMolWt              0
ExactMolWt                  0
NumValenceElectrons         0
NumRadicalElectrons         0
MaxPartialCharge            0
MinPartialCharge            0
MaxAbsPartialCharge         0
MinAbsPartialCharge         0
FpDensityMorgan1            0
FpDensityMorgan2            0
FpDensityMorgan3            0
BalabanJ                    0
BertzCT                     0
Chi0                        0
Chi0n                       0
Chi0v                       0
Chi1                        0
Chi1n                       0
Chi1v                       0
Chi2n                       0
Chi2v                       0
Chi3n                       0
Chi3v                       0
Chi4n                       0
Chi4v                       0
HallKierAlpha               0
Ipc       

In [0]:
 # Create correlation matrix
corr_matrix = df_final.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]

# Drop features 
for i in to_drop:
    df_final.drop(i, axis=1, inplace=True)

In [0]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import make_scorer 
#Normalizing
from sklearn import preprocessing

#For data split
from sklearn.model_selection import train_test_split 

#importing Random Forest packages
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import GridSearchCV

y = df_final['Censorship']
X = df_final.drop('Censorship',axis=1)

#data split based on sythesis date into train test and validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


In [77]:
clf = RFC(n_estimators=50, max_depth=6,
                             random_state=0)
clf = clf.fit(X_train, y_train)

print("The number of selected features is: {}".format(clf.n_features_))
#print(clf.feature_importances_)

The number of selected features is: 140


(686, 140)